In [26]:
import pandas as pd
import datetime as dt
import pytz
from sqlalchemy import create_engine
from tqdm import tqdm
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define your connection parameters
username = 'bullionbear'
password = 'Sunshine4Jellybean'
host = 'localhost'
port = '5432'
database = 'lynkoraDB'

# Create the connection string
connection_string = f'postgresql://{username}:{password}@{host}:{port}/{database}'
# Create the database engine
engine = create_engine(connection_string)

# Define your SQL query
# Bad data is in 2023/3/24
n_days = 366
end_time = int(dt.datetime(2024, 5, 1, tzinfo=pytz.UTC).timestamp() * 1000) - 1
start_time = end_time - n_days * 86400_000
table = "btcusdt_kline_1s"
query = f'SELECT open_time, open, high, low, close, close_time FROM {table} where open_time between {start_time} and {end_time} order by open_time asc'

# Read the data in chunks
chunksize = 86400  # Adjust the chunk size as needed
data_frames = []


for i in tqdm(range(n_days)):
    query = f"""SELECT open_time, open, high, low, close, close_time FROM {table} 
    where open_time between {start_time + i * 86400_000} and {start_time + (i + 1)*86400_000}
    order by open_time asc"""
    daily_data = pd.read_sql_query(query, engine)
    daily_data['agg'] = daily_data['open_time'] // 1800_000
    grouped = daily_data.groupby('agg').agg(
        open_time=('open_time', 'first'),
        open=('open', 'first'),
        high=('high', 'max'),
        low=('low', 'min'),
        close=('close', 'last'),
        close_time=('close_time', 'last'),
        count=('open_time', 'size')  # Count the number of records in each group
    ).reset_index(drop=True)
    data_frames.append(grouped)
    

# Combine all chunks into a single DataFrame if needed
raw = pd.concat(data_frames, ignore_index=True)

raw.head()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 366/366 [02:16<00:00,  2.69it/s]


,open_time,open,high,low,close,close_time,count
0,1682899200000,29233.20,29320.00,29215.93,29301.74,1682900999999,1800
1,1682901000000,29301.75,29337.34,29280.28,29315.57,1682902799999,1800
2,1682902800000,29315.57,29315.57,29189.15,29253.96,1682904599999,1800
3,1682904600000,29253.96,29260.04,28369.95,28504.36,1682906399999,1800
4,1682906400000,28504.36,28680.38,28414.00,28582.18,1682908199999,1800


In [27]:
df = raw.copy()
df['date'] = pd.to_datetime(df['open_time'], unit='ms')
df.set_index('date', inplace=True)
df.head()

,open_time,open,high,low,close,close_time,count
date,,,,,,,
2023-05-01 00:00:00,1682899200000,29233.20,29320.00,29215.93,29301.74,1682900999999,1800
2023-05-01 00:30:00,1682901000000,29301.75,29337.34,29280.28,29315.57,1682902799999,1800
2023-05-01 01:00:00,1682902800000,29315.57,29315.57,29189.15,29253.96,1682904599999,1800
2023-05-01 01:30:00,1682904600000,29253.96,29260.04,28369.95,28504.36,1682906399999,1800
2023-05-01 02:00:00,1682906400000,28504.36,28680.38,28414.00,28582.18,1682908199999,1800


In [28]:
df.tail()

,open_time,open,high,low,close,close_time,count
date,,,,,,,
2024-04-30 21:30:00,1714512600000,60339.27,60400.00,60166.00,60180.00,1714514399999,1800
2024-04-30 22:00:00,1714514400000,60180.00,60794.00,60156.00,60790.55,1714516199999,1800
2024-04-30 22:30:00,1714516200000,60790.54,61070.71,60447.44,60589.79,1714517999999,1800
2024-04-30 23:00:00,1714518000000,60589.79,60638.00,60401.06,60586.73,1714519799999,1800
2024-04-30 23:30:00,1714519800000,60586.73,60924.16,60562.00,60672.00,1714521599999,1800


In [25]:
df[df['count'] != 1800]

,open_time,open,high,low,close,close_time,count
date,,,,,,,
2023-03-24 12:30:00,1679661000000,28080.00,28080.00,28080.0,28080.00,1679661581646,582
2023-03-24 14:00:00,1679666400000,28079.99,28253.01,27835.0,28095.82,1679668199999,1836


In [32]:
end_time = int(dt.datetime(2024, 5, 1, tzinfo=pytz.UTC).timestamp() * 1000) - 1
start_time = end_time - n_days * 86400_000 + 1
(start_time, end_time)

(1682899200000, 1714521599999)